# Introduction

Ce notebook a pour objectif de rechercher le meilleur modèle de prédiction pour des données déjà traitées. Un des points importants qu'ilm conviendra de résoudre est le déséquilibre entre les classes. Pöur résoudre ce problème, plusieurs possibilités s'offrent à nous. Il conviendra de tester différentes méthodes mais également de tester des combinaisons de méthodes.

 Méthodes de gestion du déséquilibre de classe et un exemple :
     - choix de métriques adaptées (Score Fbeta)
     - pondération des erreurs en fonction du déséquilibre des classes (class_weight)
     - choix d'algorithmes adaptés (Random forest)
     - Sous ou sur échantillonnage (SMOTE)

Choix des métriques adaptées :
Dans le cas de classes déséquilibrées, il est fréquent que toutes les erreurs ne se valent pas. Ici, les évènements positifs (défaut de paiement) son bien plus rares que les évènements négatifs. D'un point de vue purement rentable, il vaut mieux faire l'erreur de ne pas octroyer de prêt à un bon payeur que d'en octroyer un à un mauvais payeur. Avoir des faux positifs est donc plus désirable que d'avoir des faux négatifs.

La question est plus complexe s'il on intègre des notions d'éthique, puisque refuser un prêt à un bon payeur à des conséquences négatives, ainsi il faudrait discuter des problématiques éthiques à refuser l'ocrtoi d'un prêt à un bon payeur ou en octroyer un à un mauvais payeur. Il faudrait ensuite calculer un coefficient de non-désirabilité qui permettrait d'évaluer quelle est l'importance des faux positifs par rapport aux faux négatifs.

C'est une question complexe qui demande une réflexion éthique et économique.

Pour l'exercice, nous fixerons le rapport de non désirabilité à cette règle : un faux négatif a un impact négatif 10 fois plus important qu'un faux positif.

A partir de cette règle on peut déterminer le paramètre beta d'un F-beta score qui permettra d'adapter la métrique d'évaluation au problème de déséquilibre des classes et de la différence d'impact des erreurs. Dans ce cas, c'est le recall qui aura plus d'importance que la precision. beta = racine_carrée(10)

Pondération  des erreurs :
La méthode de pondération des erreurs en fonction du déséquilibre des classes consiste à considérer les erreurs effectuées sur la classe mineure comme plus importantes que celles commises sur la classe majeure. Le ratio qui relie ces importances peut correspondre au ratio du nombre d'observations entre la classe mineure et la classe majeure.

Il me semble que cette méthode est redondante avec le choix d'un F-beta score bien que le F-beta soit conçu pour refléter la gravité d'une erreur plutôt qu'une autre alors que le class_weight est conçu pour s'adapter au déséquilibre dans le nombre d'observations.

# Stratégie 

En utilisant une métrique F-beta score pondérée pour donner plus d'importance aux faux négatifs (qui sont des erreurs plus importantes), on va tester les méthodes de sous-échantillonnage aléatoire et sur-échantillonnage type SMOTE en les combinant avec différents algorithmes tels que : la regression logistique, la random forest et kNN. Il y a d

In [1]:
pip install -U imbalanced-learn

Requirement already up-to-date: imbalanced-learn in g:\softwares\anaconda3\lib\site-packages (0.8.1)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, sys
import pandas as pd
import numpy as np
from time import time
from sklearn.model_selection import GridSearchCV

In [33]:
# Import du fichier de données pré-traitées
default_dir = "G:/OCDataScientist/Projet7"
data = pd.read_csv(os.path.join(default_dir,'data_train.csv'))

# Création d'un imputer pour remplacer les valeurs manquantes
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# Remplacement des valeurs manquantes
temp_DF = pd.DataFrame(imputer.fit_transform(data))
# Réintégration des colonnes et index
temp_DF.columns = data.columns
temp_DF.index = data.index
data = temp_DF

# Création d'un scaler 0-1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(data)

# Normalisation des variables
temp_DF = pd.DataFrame(scaler.transform(data))
# Réintégration des colonnes et index
temp_DF.columns = data.columns
temp_DF.index = data.index
data = temp_DF

# Sauvegarde du DataFrame des données prêtes à être modélisées
#Export des fichiers de données traités
output_dir = "G:/OCDataScientist/Projet7"
data.to_csv(os.path.join(output_dir,'data_train_std.csv'), index=False)


# Décomposition train/test
y = data['TARGET'].values
X = data.drop(columns = ['TARGET']).values
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3,
                                                                    random_state=0)

In [4]:
# Sous-échantillonnage
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0, replacement=True)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# Vérification
from collections import Counter
print('Répartition originale:', Counter(y_train))
print('Répartition après sous-échantillonnage', Counter(y_train_rus))

Répartition originale: Counter({0.0: 197772, 1.0: 17485})
Répartition après sous-échantillonnage Counter({0.0: 17485, 1.0: 17485})


In [5]:
# Sur-échantillonnage SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Vérification
from collections import Counter
print('Répartition originale:', Counter(y_train))
print('Répartition après sur-échantillonnage', Counter(y_train_smote))

Répartition originale: Counter({0.0: 197772, 1.0: 17485})
Répartition après sur-échantillonnage Counter({0.0: 197772, 1.0: 197772})


In [6]:
# Création d'une métrique adaptée Fbeta score pour le GridSearchCV
from sklearn import metrics
target = metrics.make_scorer(metrics.fbeta_score, beta=3.16)

In [7]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

# Entrainement sur le jeu de données déséquilibré
knn_imb = KNeighborsClassifier()
start = time()
knn_imb.fit(X_train, y_train)
end = time()
time_knn_imb_train = end - start
print("Training Time: {:.2f}".format(time_knn_imb_train))
# Prédiction
start = time()
y_knn_imb_pred = knn_imb.predict(X_test)
end = time()
time_knn_imb_pred = end - start
print("Testing Time: {:.2f}".format(time_knn_imb_pred))
# Métriques
acc_knn_imb = metrics.accuracy_score(y_test, y_knn_imb_pred)
print("Accuracy: {:.3f}".format(acc_knn_imb))
fbeta_knn_imb = metrics.fbeta_score(y_test, y_knn_imb_pred, beta=3.16)
print("Fbeta: {:.3f}".format(fbeta_knn_imb))
confusion_knn_imb = metrics.confusion_matrix(y_test, y_knn_imb_pred)
print(confusion_knn_imb)

# Entrainement sur le jeu de données sous échantillonné
knn_rus = KNeighborsClassifier()
start = time()
knn_rus.fit(X_train_rus, y_train_rus)
end = time()
time_knn_rus_train = end - start
print("Training Time: {:.2f}".format(time_knn_rus_train))
# Prédiction
start = time()
y_knn_rus_pred = knn_rus.predict(X_test)
end = time()
time_knn_rus_pred = end - start
print("Testing Time: {:.2f}".format(time_knn_rus_pred))
# Métriques
acc_knn_rus = metrics.accuracy_score(y_test, y_knn_rus_pred)
print("Accuracy: {:.3f}".format(acc_knn_rus))
fbeta_knn_rus = metrics.fbeta_score(y_test, y_knn_rus_pred, beta=3.16)
print("Fbeta: {:.3f}".format(fbeta_knn_rus))
confusion_knn_rus = metrics.confusion_matrix(y_test, y_knn_rus_pred)
print(confusion_knn_rus)

# Entrainement sur le jeu de données sur-échantillonné
knn_smote = KNeighborsClassifier()
start = time()
knn_smote.fit(X_train_smote, y_train_smote)
end = time()
time_knn_smote_train = end - start
print("Training Time: {:.2f}".format(time_knn_smote_train))
# Prédiction
start = time()
y_knn_smote_pred = knn_smote.predict(X_test)
end = time()
time_knn_smote_pred = end - start
print("Testing Time: {:.2f}".format(time_knn_smote_pred))
# Métriques
acc_knn_smote = metrics.accuracy_score(y_test, y_knn_smote_pred)
print("Accuracy: {:.3f}".format(acc_knn_smote))
fbeta_knn_smote = metrics.fbeta_score(y_test, y_knn_smote_pred, beta=3.16)
print("Fbeta: {:.3f}".format(fbeta_knn_smote))
confusion_knn_smote = metrics.confusion_matrix(y_test, y_knn_smote_pred)
print(confusion_knn_smote)

Training Time: 0.07
Testing Time: 285.29
Accuracy: 0.915
Fbeta: 0.018
[[84286   628]
 [ 7218   122]]
Training Time: 0.01
Testing Time: 47.99
Accuracy: 0.550
Fbeta: 0.412
[[46388 38526]
 [ 2987  4353]]
Training Time: 0.16
Testing Time: 553.62
Accuracy: 0.625
Fbeta: 0.354
[[54213 30701]
 [ 3879  3461]]


Sur le jeu de données déséquilibré, on retrouve la précision la plus élevée mais on remarque qu'il ne parvient à détecter seulement 122 TP ce qui est très faible. Le Fbeta résultant qui met l'accent sur les TP est donc très bas. Ici le problème du déséquilibre de classes est flagrant.

Sur le jeu sous échantillonné, la précision est la plus faible, il repère en effet beaucoup de FP, cependant c'est sur ce jeu là que l'on trouve le plus de TP, il obtient donc le Fbeta le plus élevé.

Sur le jeu sur-échantillonné, on obtient des performances plus faibles sur le nombre de TP, il échoue à détecter plus de la moitié des positifs.

Globalement, si le jeu sous-échantillonné s'en sort mieux (en prenant comme objectif Fbeta avec beta=3.16), le modèle KNN présente des performances faibles, il repère au mieux 59% des positifs avec un taux de FP de presque 10%.

Pour la suite de la recherche du meilleur algorithme et pour optimiser la recherche des hyperparamètres, il serait intéressant de réduire la taille de l'échantillon et s'il faut réduire l'échantillon pour accélérer cette phase, alors autant en profiter pour le rééquilibrer. La recherche de l'algorithme et des hyper paramèrtres optimaux se fera donc avec le jeu de données rééquilibré par échantillonnage.

In [10]:
# Regression logistique avec GridSearchCV
from sklearn.linear_model import LogisticRegression
paramGrid = {'penalty': ['l1','l2','elasticnet'], 'random_state' : [0],
             'class_weight' : [None], 
             'solver' : ['newton-cg', 'lbfgs', 'liblinear','sag','saga'], 'max_iter' :[1000], 'l1_ratio' : [0.5] }

# Entrainement sur le jeu de données sous échantillonné
start = time()
reg_rus = reg = model_selection.GridSearchCV(LogisticRegression(), paramGrid, cv=5, scoring=target)
reg_rus.fit(X_train_rus, y_train_rus)
end = time()
time_reg_grid = end - start
print("Testing Time: {:.2f}".format(time_reg_grid))

print(reg_rus.best_params_)
# Afficher les performances correspondantes
print("Résultats de la validation croisée :")
for mean, std, params in zip(reg_rus.cv_results_['mean_test_score'],
                             reg_rus.cv_results_['std_test_score'],  reg_rus.cv_results_['params']):
    print("{} = {:.3f} (+/-{:.03f}) for {}".format(target,mean,std,params))

G:\Softwares\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
G:\Softwares\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
G:\Softwares\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
G:\Softwares\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
G:\Softwares\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
G:\Softwares\Anaconda3\lib\site-packages\sklearn\l

Testing Time: 453.56
{'class_weight': None, 'l1_ratio': 0.5, 'max_iter': 1000, 'penalty': 'l2', 'random_state': 0, 'solver': 'liblinear'}
Résultats de la validation croisée :
make_scorer(fbeta_score, beta=3.16) = nan (+/-nan) for {'class_weight': None, 'l1_ratio': 0.5, 'max_iter': 1000, 'penalty': 'l1', 'random_state': 0, 'solver': 'newton-cg'}
make_scorer(fbeta_score, beta=3.16) = nan (+/-nan) for {'class_weight': None, 'l1_ratio': 0.5, 'max_iter': 1000, 'penalty': 'l1', 'random_state': 0, 'solver': 'lbfgs'}
make_scorer(fbeta_score, beta=3.16) = 0.677 (+/-0.009) for {'class_weight': None, 'l1_ratio': 0.5, 'max_iter': 1000, 'penalty': 'l1', 'random_state': 0, 'solver': 'liblinear'}
make_scorer(fbeta_score, beta=3.16) = nan (+/-nan) for {'class_weight': None, 'l1_ratio': 0.5, 'max_iter': 1000, 'penalty': 'l1', 'random_state': 0, 'solver': 'sag'}
make_scorer(fbeta_score, beta=3.16) = 0.677 (+/-0.009) for {'class_weight': None, 'l1_ratio': 0.5, 'max_iter': 1000, 'penalty': 'l1', 'random_s

Le meilleur solver par regression logistique semble être le liblinear avec une pénalité l2. Nous allons maintenant le tester avec les 3 jeux de données.

In [14]:
# Regression Logistique

# Entrainement sur le jeu de données déséquilibré
reg_imb = LogisticRegression(penalty = 'l2', solver = 'liblinear', random_state = 0, max_iter = 1000)
start = time()
reg_imb.fit(X_train, y_train)
end = time()
time_reg_imb_train = end - start
print("Training Time: {:.2f}".format(time_reg_imb_train))
# Prédiction
start = time()
y_reg_imb_pred = reg_imb.predict(X_test)
end = time()
time_reg_imb_pred = end - start
print("Testing Time: {:.2f}".format(time_reg_imb_pred))
# Métriques
acc_reg_imb = metrics.accuracy_score(y_test, y_reg_imb_pred)
print("Accuracy: {:.3f}".format(acc_reg_imb))
fbeta_reg_imb = metrics.fbeta_score(y_test, y_reg_imb_pred, beta=3.16)
print("Fbeta: {:.3f}".format(fbeta_reg_imb))
confusion_reg_imb = metrics.confusion_matrix(y_test, y_reg_imb_pred)
print(confusion_reg_imb)

# Entrainement sur le jeu de données sous échantillonné
reg_rus = LogisticRegression(penalty = 'l2', solver = 'liblinear', random_state = 0, max_iter = 1000)
start = time()
reg_rus.fit(X_train_rus, y_train_rus)
end = time()
time_reg_rus_train = end - start
print("Training Time: {:.2f}".format(time_reg_rus_train))
# Prédiction
start = time()
y_reg_rus_pred = reg_rus.predict(X_test)
end = time()
time_reg_rus_pred = end - start
print("Testing Time: {:.2f}".format(time_reg_rus_pred))
# Métriques
acc_reg_rus = metrics.accuracy_score(y_test, y_reg_rus_pred)
print("Accuracy: {:.3f}".format(acc_reg_rus))
fbeta_reg_rus = metrics.fbeta_score(y_test, y_reg_rus_pred, beta=3.16)
print("Fbeta: {:.3f}".format(fbeta_reg_rus))
confusion_reg_rus = metrics.confusion_matrix(y_test, y_reg_rus_pred)
print(confusion_reg_rus)

# Entrainement sur le jeu de données sur-échantillonné
reg_smote = LogisticRegression(penalty = 'l2', solver = 'liblinear', random_state = 0, max_iter = 1000)
start = time()
reg_smote.fit(X_train_smote, y_train_smote)
end = time()
time_reg_smote_train = end - start
print("Training Time: {:.2f}".format(time_reg_smote_train))
# Prédiction
start = time()
y_reg_smote_pred = reg_smote.predict(X_test)
end = time()
time_reg_smote_pred = end - start
print("Testing Time: {:.2f}".format(time_reg_smote_pred))
# Métriques
acc_reg_smote = metrics.accuracy_score(y_test, y_reg_smote_pred)
print("Accuracy: {:.3f}".format(acc_reg_smote))
fbeta_reg_smote = metrics.fbeta_score(y_test, y_reg_smote_pred, beta=3.16)
print("Fbeta: {:.3f}".format(fbeta_reg_smote))
confusion_reg_smote = metrics.confusion_matrix(y_test, y_reg_smote_pred)
print(confusion_reg_smote)

Training Time: 13.45
Testing Time: 0.03
Accuracy: 0.920
Fbeta: 0.012
[[84821    93]
 [ 7262    78]]
Training Time: 1.98
Testing Time: 0.03
Accuracy: 0.685
Fbeta: 0.523
[[58212 26702]
 [ 2344  4996]]
Training Time: 36.35
Testing Time: 0.03
Accuracy: 0.698
Fbeta: 0.507
[[59668 25246]
 [ 2574  4766]]


La comparaison des jeux de données est assez similaire avec celle que nous avons fait lors du kNN, le jeu déséquilibré apporte une précision très élevée mais ne parvient pas à détecter les TP, le jeu sous-échantillonné présente les meilleurs résultats sur la métrique principale (Fbeta score).

Si la regression logistique présente de meilleurs résultats que le kNN, notamment sur une bonne détection des TP, elle échoue en détectant près d'un tiers de faux positifs. Au vu de l'utilisation finale du modèle, ces résultats ne me semblent pas suffisants.

In [19]:
# Random forest avec GridSearch CV
from sklearn.ensemble import GradientBoostingClassifier
# Grilles dans l'ordre de recherche
paramGrid1 = {'loss' : ['deviance', 'exponential'],
             'max_depth': [3,4,5], 'learning_rate':[0.05,0.1,0.02] ,  'n_estimators':[50,100,200], 
             'random_state':[0]}
paramGrid2 = {'loss' : ['exponential'], 'learning_rate':[0.08,0.1,0.12] ,  'n_estimators':[150,200,250], 
             'random_state':[0]}
paramGrid3 = {'loss' : ['exponential'], 'learning_rate': [0.12, 0.15, 0.25] ,  'n_estimators':[250,300,400], 
             'random_state':[0]}

# Entrainement sur le jeu de données sous echantillonné
start = time()
rf_rus = model_selection.GridSearchCV(GradientBoostingClassifier(), paramGrid3, cv=5, scoring=target)
rf_rus.fit(X_train_rus, y_train_rus)
end = time()
time_rf_grid = end - start
print("Testing Time: {:.2f}".format(time_rf_grid))

print(rf_rus.best_params_)
# Afficher les performances correspondantes
print("Résultats de la validation croisée :")
for mean, std, params in zip(rf_rus.cv_results_['mean_test_score'], 
                             rf_rus.cv_results_['std_test_score'], rf_rus.cv_results_['params']):
    print("{} = {:.3f} (+/-{:.03f}) for {}".format(target,mean,std,params))

Testing Time: 5289.83
{'learning_rate': 0.12, 'loss': 'exponential', 'n_estimators': 250, 'random_state': 0}
Résultats de la validation croisée :
make_scorer(fbeta_score, beta=3.16) = 0.693 (+/-0.009) for {'learning_rate': 0.12, 'loss': 'exponential', 'n_estimators': 250, 'random_state': 0}
make_scorer(fbeta_score, beta=3.16) = 0.692 (+/-0.007) for {'learning_rate': 0.12, 'loss': 'exponential', 'n_estimators': 300, 'random_state': 0}
make_scorer(fbeta_score, beta=3.16) = 0.691 (+/-0.007) for {'learning_rate': 0.12, 'loss': 'exponential', 'n_estimators': 400, 'random_state': 0}
make_scorer(fbeta_score, beta=3.16) = 0.691 (+/-0.007) for {'learning_rate': 0.15, 'loss': 'exponential', 'n_estimators': 250, 'random_state': 0}
make_scorer(fbeta_score, beta=3.16) = 0.690 (+/-0.004) for {'learning_rate': 0.15, 'loss': 'exponential', 'n_estimators': 300, 'random_state': 0}
make_scorer(fbeta_score, beta=3.16) = 0.692 (+/-0.007) for {'learning_rate': 0.15, 'loss': 'exponential', 'n_estimators': 40

En première approche, on a trouvé les meilleurs paramètres tels que : {'learning_rate': 0.1, 'loss': 'exponential', 'max_depth': 4, 'n_estimators': 200, 'random_state': 0}

En deuxième approche, {'learning_rate': 0.12, 'loss': 'exponential', 'max_depth': 4, 'n_estimators': 250, 'random_state': 0}

En troisième approche, {'learning_rate': 0.12, 'loss': 'exponential', 'max_depth': 4, 'n_estimators': 250, 'random_state': 0} 

Les mêmes paramètres qu'en deuxième approche.

Nous allons maintenant tester ces paramètres avec les 3 jeux de données.

In [20]:
# Random Forest avec Gradient Boosting

# Entrainement sur le jeu de données déséquilibré
rf_imb = GradientBoostingClassifier(learning_rate = 0.12, loss = 'exponential', max_depth = 4, 
                                    n_estimators = 250, random_state = 0)
start = time()
rf_imb.fit(X_train, y_train)
end = time()
time_rf_imb_train = end - start
print("Training Time: {:.2f}".format(time_rf_imb_train))
# Prédiction
start = time()
y_rf_imb_pred = rf_imb.predict(X_test)
end = time()
time_rf_imb_pred = end - start
print("Testing Time: {:.2f}".format(time_rf_imb_pred))
# Métriques
acc_rf_imb = metrics.accuracy_score(y_test, y_rf_imb_pred)
print("Accuracy: {:.3f}".format(acc_rf_imb))
fbeta_rf_imb = metrics.fbeta_score(y_test, y_rf_imb_pred, beta=3.16)
print("Fbeta: {:.3f}".format(fbeta_rf_imb))
confusion_rf_imb = metrics.confusion_matrix(y_test, y_rf_imb_pred)
print(confusion_rf_imb)

# Entrainement sur le jeu de données sous échantillonné
rf_rus = GradientBoostingClassifier(learning_rate = 0.12, loss = 'exponential', max_depth = 4, 
                                    n_estimators = 250, random_state = 0)
start = time()
rf_rus.fit(X_train_rus, y_train_rus)
end = time()
time_rf_rus_train = end - start
print("Training Time: {:.2f}".format(time_rf_rus_train))
# Prédiction
start = time()
y_rf_rus_pred = rf_rus.predict(X_test)
end = time()
time_rf_rus_pred = end - start
print("Testing Time: {:.2f}".format(time_rf_rus_pred))
# Métriques
acc_rf_rus = metrics.accuracy_score(y_test, y_rf_rus_pred)
print("Accuracy: {:.3f}".format(acc_rf_rus))
fbeta_rf_rus = metrics.fbeta_score(y_test, y_rf_rus_pred, beta=3.16)
print("Fbeta: {:.3f}".format(fbeta_rf_rus))
confusion_rf_rus = metrics.confusion_matrix(y_test, y_rf_rus_pred)
print(confusion_rf_rus)

# Entrainement sur le jeu de données sur-échantillonné
rf_smote = GradientBoostingClassifier(learning_rate = 0.12, loss = 'exponential', max_depth = 4, 
                                    n_estimators = 250, random_state = 0)
start = time()
rf_smote.fit(X_train_smote, y_train_smote)
end = time()
time_rf_smote_train = end - start
print("Training Time: {:.2f}".format(time_rf_smote_train))
# Prédiction
start = time()
y_rf_smote_pred = rf_smote.predict(X_test)
end = time()
time_rf_smote_pred = end - start
print("Testing Time: {:.2f}".format(time_rf_smote_pred))
# Métriques
acc_rf_smote = metrics.accuracy_score(y_test, y_rf_smote_pred)
print("Accuracy: {:.3f}".format(acc_rf_smote))
fbeta_rf_smote = metrics.fbeta_score(y_test, y_rf_smote_pred, beta=3.16)
print("Fbeta: {:.3f}".format(fbeta_rf_smote))
confusion_rf_smote = metrics.confusion_matrix(y_test, y_rf_smote_pred)
print(confusion_rf_smote)

Training Time: 1194.85
Testing Time: 0.96
Accuracy: 0.921
Fbeta: 0.025
[[84765   149]
 [ 7174   166]]
Training Time: 150.19
Testing Time: 0.83
Accuracy: 0.694
Fbeta: 0.534
[[58923 25991]
 [ 2266  5074]]
Training Time: 2730.11
Testing Time: 0.86
Accuracy: 0.920
Fbeta: 0.035
[[84640   274]
 [ 7104   236]]


Sur le modèle de forêt aléatoire avec boost par gradient, on obtient toujours un meilleur résultat avec la méthode de sous-échantillonnage, malgré la perte d'information que cette dernière induit. Avec un score de 0.534, il est même légèrement supérieur à la méthode de regression logistique, malgré la perte d'information que cette dernière induit.
Si l'on se réfère à la matrice de confusion, on a 27% de FP ce qui est un point négatif non négligeable mais on réussi à repérer 69% de TP ce qui est plutôt élevé considérant que ce n'est pas un problème aisé.
Pour ces raisons, c'est ce modèle que l'on utilisera par la suite pour l'API.

In [37]:
# Persistance du modèle
import joblib
joblib.dump(rf_rus, 'model_rf.pkl')

['model_rf.pkl']